### Cine21 영화배우 정보 크롤링
* site 주소
    * http://www.cine21.com/rank/person/
* 개발자 도구에서 XHR인 것만 필터링 network -> content 페이지의 요청방식 확인
* Request URL : http://www.cine21.com/rank/person/content
* Request Method : POST
* From Data (1개월치 데이터)
    - section: actor
    - period_start: 2020-07
    - gender: all
    - page: 1
* 1~2 년치 데이터를 한꺼번에 크롤링 해서 actor.json파일로 저장하기

In [20]:
from bs4 import BeautifulSoup
import requests
import re
import pymongo
from urllib.parse import urljoin

---
### 정규표현식
* \w 는 문자와 숫자
* [xy] : x,y중 하나이상을 찾습니다.
* re.complie(r'[xy]') 는 해당 정규표현식을 찾아주는 컴파일러를 생성해주는 함수
    * complie 함수 안에 정규표현식 앞에는 항상 r을 붙인다

* * : 0 ~ n (횟수), + : 1 ~ n, ? : 0(zero) or 1
* .(period) : 줄바꿈 문자(\n)을 제외한 모든 글자 1개를 의미한다
* .* 는 문자가 0번 또는 그 이상 반복되는 패턴
* Greedy(.*) vs Non-Greedy(.*?)
    * Greedy : 모든 태그가 선택됨
    * Non-Greedy : 첫번째 매칭되는 태그만 선택됨
    * ![non_greedy](mongoDB_pymongo/5/5_1.PNG)
---

In [62]:
actor_url = 'http://www.cine21.com/rank/person/content'

formdata = dict()
formdata['section'] = 'actor'
formdata['period_start'] = '2020-07'
formdata['gender'] = 'all'
formdata['page'] = 1

res = requests.post(actor_url, data=formdata)
print(res.ok)
soup = BeautifulSoup(res.text, 'html.parser')
len(soup)

True


4

---
### 1단계 : 세부 페이지의 배우 정보 담기

In [55]:
actor_item_list = list() # 세부 페이지의 배우정보를 담을 리스트

for actor in soup.select('li.people_li div.name'):
#     print(actor.text)
    print(re.sub('\(\w*\)' , '' , actor.text)) ## 이름
    actor_detail_url = urljoin(actor_url,actor.select_one('a')['href'])
    print(actor_detail_url) ## 세부 페이지 url
    
    ### 세부 페이지 post  ###
    res2 = requests.post(actor_detail_url, data=formdata)
#     print(res2.ok)
    soup2 = BeautifulSoup(res2.text, 'html.parser')
    
    
    ### 세부 페이지 크롤링 ###
    actor_item_dict = dict()
    for li_tag in soup2.select('ul.default_info li'):
#         print(li_tag)
        actor_item_field = li_tag.select_one('span.tit').text
#         print(actor_item_field) # 항목의 제목을 가져옴
        
        # 정규표현식을 사용하여 item_value를 가져옴
        actor_item_value = re.sub('<span.*?>.*?</span>','', str(li_tag))
        actor_item_value = re.sub('<.*?>','',actor_item_value)
        regex = re.compile(r'[\n\t\r]')
        actor_item_value = regex.sub('',actor_item_value)
#         print(actor_item_value)
        
        actor_item_dict[actor_item_field] = actor_item_value
#     print(actor_item_dict,'\n')
    actor_item_list.append(actor_item_dict)
    

actor_item_list
        


강동원
http://www.cine21.com/db/person/info/?person_id=18040
이정현
http://www.cine21.com/db/person/info/?person_id=2760
이레
http://www.cine21.com/db/person/info/?person_id=87306
권해효
http://www.cine21.com/db/person/info/?person_id=167
구교환
http://www.cine21.com/db/person/info/?person_id=71787
김민재
http://www.cine21.com/db/person/info/?person_id=40151
유아인
http://www.cine21.com/db/person/info/?person_id=57657


[{'직업': '배우',
  '생년월일': '1981-01-18',
  '성별': '남',
  '홈페이지': 'http://www.gangdongwon.com',
  '신장/체중': '186cm, 68kg',
  '학교': '한양대 - 상명대학교대학원 연극영화 (석사과정 재학)',
  '취미': '음악감상, 웹서핑, 운동, 신발 모으기',
  '특기': '축구'},
 {'직업': '배우',
  '생년월일': '1980-02-07',
  '성별': '여',
  '홈페이지': 'https://www.instagram.com/mermaidleejh/https://twitter.com/mermaidjh',
  '신장/체중': '158cm, 38kg',
  '학교': '중앙대학교 연극영화',
  '취미': '바비 인형 수집, 피아노 연주',
  '특기': '별관찰, 시나리오 쓰기',
  '소속사': '에이바 앤 엔터테인먼트'},
 {'직업': '배우',
  '생년월일': '2006-03-12',
  '성별': '여',
  '홈페이지': 'https://www.facebook.com/leere.vv/'},
 {'직업': '배우',
  '생년월일': '1965-11-06',
  '성별': '남',
  '신장/체중': '170cm, 62kg',
  '학교': '한양대학교 연극영화학 졸업'},
 {'직업': '배우', '생년월일': '1982-12-14', '성별': '남'},
 {'직업': '배우',
  '생년월일': '1979-03-20',
  '성별': '남',
  '홈페이지': 'https://www.facebook.com/cjesminjae',
  '학교': '한국예술종합학교'},
 {'다른 이름': '엄홍식',
  '직업': '배우',
  '생년월일': '1986-10-06',
  '성별': '남',
  '홈페이지': 'https://www.instagram.com/hongsick/https://twitter.com/seeksik',
  '신장/체중': '178cm

---
#### 정규표현식 test

In [15]:
# 정규표현식을 사용하여 원하는 text 뽑아내기
test_date = '강동원(1편)'
re.sub('\(\w*\)' , '' , test_date)

'강동원'

---

In [43]:
test_data = '''
홈페이지

https://www.facebook.com/cjesminjae

'''

regex = re.compile(r'[\n\r\t]')
regex.sub('', test_data)

'홈페이지https://www.facebook.com/cjesminjae'

---
### 2단계 : 메인 페이지의 이름과 흥행지수 담기
* 1page 에 있는 7명의 배우 정보를 리스트로 만들기

In [86]:
for actor in soup.select('ul.mov_list > li > a'):
    mov_list = actor.select('span')
    print(mov_list, '\n')

[<span>반도</span>] 

[<span>반도</span>] 

[<span>반도</span>] 

[<span>반도</span>] 

[<span>사이비</span>] 

[<span>한낮의 피크닉</span>] 

[<span>반도</span>] 

[<span>반도</span>] 

[<span>＃살아있다</span>] 

[<span>버닝</span>] 



In [103]:
actor_item_list = list() # 배우의 모든 정보를 담을 리스트 (이름, 흥행지수, 출연작, 등)

actors = soup.select('li.people_li div.name')
hits = soup.select('ul.num_info > li > strong') # 흥행지수
movies = soup.select('ul.mov_list') # 영화목록
rank = soup.select('span.grade')

for idx, actor in enumerate(actors):
#     print(actor)
    # 강동원(1편) 에서 (1편)을 제거하고 저장한다.
    actor_name = re.sub('\(\w*\)' , '' , actor.text) # 배우 이름
    
    # idx번째 배우의 흥행지수
    actor_hit = int(hits[idx].text.replace(',','')) ## (,)를 제거하고 숫자로 변환
#     print(actor_name, actor_hit) # 이름과 흥행지수
    
    # 출연작
    movies_titles = movies[idx].select('li a span')
    movie_title_list = list() # 출연작 리스트
    for m in movies_titles:
        movie_title_list.append(m.text)
#     print(movies_titles)
    
    # 순위
    ranking = int(rank[idx].text)
#     print(ranking)
    
    # 세부 페이지 url
    actor_detail_url = urljoin(actor_url,actor.select_one('a')['href'])
#     print(actor_detail_url)
    
    
    ### 세부 페이지 post  ###
    res2 = requests.post(actor_detail_url, data=formdata)
#     print(res2.ok)
    soup2 = BeautifulSoup(res2.text, 'html.parser')
    
    
    ## 1명의 배우에 대한 dict
    actor_item_dict = dict()
    
    actor_item_dict['배우이름'] = actor_name
    actor_item_dict['흥행지수'] = actor_hit
    actor_item_dict['출연작'] = movie_title_list
    actor_item_dict['순위'] = ranking
    
    ### 세부 페이지 크롤링 ###
    for li_tag in soup2.select('ul.default_info li'):
#         print(li_tag)
        actor_item_field = li_tag.select_one('span.tit').text
#         print(actor_item_field) # 항목의 제목을 가져옴
        
        # 정규표현식을 사용하여 item_value를 가져옴
        actor_item_value = re.sub('<span.*?>.*?</span>','', str(li_tag))
        actor_item_value = re.sub('<.*?>','',actor_item_value)
        regex = re.compile(r'[\n\t\r]')
        actor_item_value = regex.sub('',actor_item_value)
#         print(actor_item_value)
        
        actor_item_dict[actor_item_field] = actor_item_value
#     print(actor_item_dict,'\n')
    actor_item_list.append(actor_item_dict)

actor_item_list


[{'배우이름': '강동원',
  '흥행지수': 159665,
  '출연작': ['반도'],
  '순위': 1,
  '직업': '배우',
  '생년월일': '1981-01-18',
  '성별': '남',
  '홈페이지': 'http://www.gangdongwon.com',
  '신장/체중': '186cm, 68kg',
  '학교': '한양대 - 상명대학교대학원 연극영화 (석사과정 재학)',
  '취미': '음악감상, 웹서핑, 운동, 신발 모으기',
  '특기': '축구'},
 {'배우이름': '이정현',
  '흥행지수': 139707,
  '출연작': ['반도'],
  '순위': 2,
  '직업': '배우',
  '생년월일': '1980-02-07',
  '성별': '여',
  '홈페이지': 'https://www.instagram.com/mermaidleejh/https://twitter.com/mermaidjh',
  '신장/체중': '158cm, 38kg',
  '학교': '중앙대학교 연극영화',
  '취미': '바비 인형 수집, 피아노 연주',
  '특기': '별관찰, 시나리오 쓰기',
  '소속사': '에이바 앤 엔터테인먼트'},
 {'배우이름': '이레',
  '흥행지수': 119748,
  '출연작': ['반도'],
  '순위': 3,
  '직업': '배우',
  '생년월일': '2006-03-12',
  '성별': '여',
  '홈페이지': 'https://www.facebook.com/leere.vv/'},
 {'배우이름': '권해효',
  '흥행지수': 99795,
  '출연작': ['반도', '사이비', '한낮의 피크닉'],
  '순위': 4,
  '직업': '배우',
  '생년월일': '1965-11-06',
  '성별': '남',
  '신장/체중': '170cm, 62kg',
  '학교': '한양대학교 연극영화학 졸업'},
 {'배우이름': '구교환',
  '흥행지수': 79832,
  '출연작': ['반도'],
  '순위': 5,
 

---
### 3단계 : 여러 페이지 크롤링 하기
* 3page 까지 크롤링 하기
* 한 페이지에 7명씩 21명의 배우를 크롤링

In [109]:
actor_url = 'http://www.cine21.com/rank/person/content'

formdata = dict()
formdata['section'] = 'actor'
formdata['period_start'] = '2020-07'
formdata['gender'] = 'all'

actor_item_list = list() # 배우의 모든 정보를 담을 리스트 (이름, 흥행지수, 출연작, 등)

for page in range(1,4):
    formdata['page'] = page
    
    print('==========={} Page============================================='.format(page))

    res = requests.post(actor_url, data=formdata)
#     print(res.ok)
    soup = BeautifulSoup(res.text, 'html.parser')
#     len(soup)



    

    actors = soup.select('li.people_li div.name')
    hits = soup.select('ul.num_info > li > strong') # 흥행지수
    movies = soup.select('ul.mov_list') # 영화목록
    rank = soup.select('span.grade')

    for idx, actor in enumerate(actors):
    #     print(actor)
        # 강동원(1편) 에서 (1편)을 제거하고 저장한다.
        print('===> {} 번째 배우 출력'.format(idx))
        actor_name = re.sub('\(\w*\)' , '' , actor.text) # 배우 이름

        # idx번째 배우의 흥행지수
        actor_hit = int(hits[idx].text.replace(',','')) ## (,)를 제거하고 숫자로 변환
    #     print(actor_name, actor_hit) # 이름과 흥행지수

        # 출연작
        movies_titles = movies[idx].select('li a span')
        movie_title_list = list() # 출연작 리스트
        for m in movies_titles:
            movie_title_list.append(m.text)
    #     print(movies_titles)

        # 순위
        ranking = int(rank[idx].text)
    #     print(ranking)

        # 세부 페이지 url
        actor_detail_url = urljoin(actor_url,actor.select_one('a')['href'])
    #     print(actor_detail_url)


        ### 세부 페이지 post  ###
        res2 = requests.post(actor_detail_url, data=formdata)
    #     print(res2.ok)
        soup2 = BeautifulSoup(res2.text, 'html.parser')


        ## 1명의 배우에 대한 dict
        actor_item_dict = dict()

        actor_item_dict['배우이름'] = actor_name
        actor_item_dict['흥행지수'] = actor_hit
        actor_item_dict['출연작'] = movie_title_list
        actor_item_dict['순위'] = ranking

        ### 세부 페이지 크롤링 ###
        for li_tag in soup2.select('ul.default_info li'):
    #         print(li_tag)
            actor_item_field = li_tag.select_one('span.tit').text
    #         print(actor_item_field) # 항목의 제목을 가져옴

            # 정규표현식을 사용하여 item_value를 가져옴
            actor_item_value = re.sub('<span.*?>.*?</span>','', str(li_tag))
            actor_item_value = re.sub('<.*?>','',actor_item_value)
            regex = re.compile(r'[\n\t\r]')
            actor_item_value = regex.sub('',actor_item_value)
    #         print(actor_item_value)

            actor_item_dict[actor_item_field] = actor_item_value
    #     print(actor_item_dict,'\n')
        actor_item_list.append(actor_item_dict)
        print(actor_item_dict)

len(actor_item_list)


===========1 Page=============================================
===> 0 번째 배우 출력
{'배우이름': '강동원', '흥행지수': 159665, '출연작': ['반도'], '순위': 1, '직업': '배우', '생년월일': '1981-01-18', '성별': '남', '홈페이지': 'http://www.gangdongwon.com', '신장/체중': '186cm, 68kg', '학교': '한양대 - 상명대학교대학원 연극영화 (석사과정 재학)', '취미': '음악감상, 웹서핑, 운동, 신발 모으기', '특기': '축구'}
===> 1 번째 배우 출력
{'배우이름': '이정현', '흥행지수': 139707, '출연작': ['반도'], '순위': 2, '직업': '배우', '생년월일': '1980-02-07', '성별': '여', '홈페이지': 'https://www.instagram.com/mermaidleejh/https://twitter.com/mermaidjh', '신장/체중': '158cm, 38kg', '학교': '중앙대학교 연극영화', '취미': '바비 인형 수집, 피아노 연주', '특기': '별관찰, 시나리오 쓰기', '소속사': '에이바 앤 엔터테인먼트'}
===> 2 번째 배우 출력
{'배우이름': '이레', '흥행지수': 119748, '출연작': ['반도'], '순위': 3, '직업': '배우', '생년월일': '2006-03-12', '성별': '여', '홈페이지': 'https://www.facebook.com/leere.vv/'}
===> 3 번째 배우 출력
{'배우이름': '권해효', '흥행지수': 99795, '출연작': ['반도', '사이비', '한낮의 피크닉'], '순위': 4, '직업': '배우', '생년월일': '1965-11-06', '성별': '남', '신장/체중': '170cm, 62kg', '학교': '한양대학교 연극영화학 졸업'}
===> 4 번째 배우 출력
{'배우

21

---
### 4단계 : 1년치 데이터 크롤링 해보기
* 273page 까지 크롤링 하기
* 한 페이지에 7명씩 1905명의 배우를 크롤링

In [111]:
from bs4 import BeautifulSoup
import requests
import re
import pymongo
from urllib.parse import urljoin
from itertools import count # count itertools 사용

In [120]:
actor_url = 'http://www.cine21.com/rank/person/content'

formdata = dict()
formdata['section'] = 'actor'
formdata['period_start'] = '2019-07'
formdata['gender'] = 'all'

actor_item_list = list() # 배우의 모든 정보를 담을 리스트 (이름, 흥행지수, 출연작, 등)

for page in range(1,274):
    formdata['page'] = page
    
    print('==========={} Page============================================='.format(page))

    res = requests.post(actor_url, data=formdata)
#     print(res.ok)
    soup = BeautifulSoup(res.text, 'html.parser')
#     len(soup)



    actors = soup.select('li.people_li div.name')
    if len(actors) == 0:  # 잘못된 정보는 걸러내기
        break
    
    hits = soup.select('ul.num_info > li > strong') # 흥행지수
    movies = soup.select('ul.mov_list') # 영화목록
    rank = soup.select('span.grade')

    for idx, actor in enumerate(actors):
    #     print(actor)
        # 강동원(1편) 에서 (1편)을 제거하고 저장한다.
        actor_name = re.sub('\(\w*\)' , '' , actor.text) # 배우 이름
        print(' ===> {}({}) 배우'.format(actor_name,len(actor_name)))

        # idx번째 배우의 흥행지수
        actor_hit = int(hits[idx].text.replace(',','')) ## (,)를 제거하고 숫자로 변환
    #     print(actor_name, actor_hit) # 이름과 흥행지수

        # 출연작
        movies_titles = movies[idx].select('li a span')
        movie_title_list = list() # 출연작 리스트
        for m in movies_titles:
            movie_title_list.append(m.text)
    #     print(movies_titles)

        # 순위
        ranking = int(rank[idx].text)
    #     print(ranking)

        # 세부 페이지 url
        actor_detail_url = urljoin(actor_url,actor.select_one('a')['href'])
    #     print(actor_detail_url)


        ### 세부 페이지 post  ###
        res2 = requests.post(actor_detail_url, data=formdata)
    #     print(res2.ok)
        soup2 = BeautifulSoup(res2.text, 'html.parser')


        ## 1명의 배우에 대한 dict
        actor_item_dict = dict()

        actor_item_dict['배우이름'] = actor_name
        actor_item_dict['흥행지수'] = actor_hit
        actor_item_dict['출연작'] = movie_title_list
        actor_item_dict['순위'] = ranking

        ### 세부 페이지 크롤링 ###
        for li_tag in soup2.select('ul.default_info li'):
    #         print(li_tag)
            actor_item_field = li_tag.select_one('span.tit').text
    #         print(actor_item_field) # 항목의 제목을 가져옴

            # 정규표현식을 사용하여 item_value를 가져옴
            actor_item_value = re.sub('<span.*?>.*?</span>','', str(li_tag))
            actor_item_value = re.sub('<.*?>','',actor_item_value)
            regex = re.compile(r'[\n\t\r]')
            actor_item_value = regex.sub('',actor_item_value)
    #         print(actor_item_value)

            actor_item_dict[actor_item_field] = actor_item_value
    #     print(actor_item_dict,'\n')
        actor_item_list.append(actor_item_dict)
#         print(actor_item_dict)

len(actor_item_list)


===========273 Page=============================================
 ===> 윤지원(3) 배우


1905

In [121]:
print(len(actor_item_list))
actor_final_list = [ actor for actor in actor_item_list if len(actor['배우이름'])<10]
print(len(actor_final_list))

1905
1903


In [124]:
actor_item_list[:2]

[{'배우이름': '이병헌',
  '흥행지수': 45760,
  '출연작': ['백두산', '남산의 부장들', '내부자들', '그것만이 내 세상', '광해, 왕이 된 남자', '번지점프를 하다'],
  '순위': 1,
  '다른 이름': 'Byung-hun Lee;BH Lee',
  '직업': '배우',
  '생년월일': '1970-07-12',
  '성별': '남',
  '홈페이지': 'http://www.leebyunghun.kr/',
  '신장/체중': '177cm, 72kg',
  '학교': '한양대학교 불어문학과',
  '취미': '모자수집, 여행',
  '특기': '태권도, 스노우보드, 수영, 팔씨름'},
 {'배우이름': '하정우',
  '흥행지수': 31124,
  '출연작': ['백두산', '클로젯', '멋진 하루', '신과 함께-인과 연', '신과 함께-죄와 벌', '아가씨'],
  '순위': 2,
  '다른 이름': '김성훈; 河正宇',
  '직업': '배우',
  '생년월일': '1978-03-11',
  '성별': '남',
  '홈페이지': 'https://www.facebook.com/ft.hajungwoo',
  '신장/체중': '184cm, 75kg',
  '학교': '중앙대학교 연극학 학사',
  '취미': '피아노, 검도, 수영'}]

---
### 5단계 : json 파일로 저장하기

In [130]:
import json

with open('data/actors.json', 'w', encoding='utf8') as file:
    json.dump(actor_final_list, file,ensure_ascii=False)
    print('file 저장 완료')

file 저장 완료
